In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium, GCNNModelLarge
from classes.models import LinearClassifier
from datasets import load_cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar100(augment=True)

CIFAR-100 loaded in 100 seconds


In [3]:
def test_model(feature_model, settings, data, num_features, num_classes, feature_samplers, classification_samplers,
               f_num_steps, f_lr, f_keep_prob, f_reg, c_num_steps, c_lr, c_reg):
    
    tf.reset_default_graph()
    
    f_train, f_valid, f_test = feature_samplers
    c_train, c_valid, c_test = classification_samplers
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])    
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    f_model = feature_model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=f_lr, reg=f_reg, dropout=dropout, training=training)
    
    features = tf.placeholder(tf.float32, shape=[None, num_features])
    c_model = LinearClassifier(x=features, y=y, num_features=num_features, num_classes=num_features, 
                               lr=c_lr, reg=c_reg)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g, dropout: %g" %(num_features, f_lr, f_reg, f_keep_prob))
        train_time = time.time()
        for step in range(f_num_steps):        
            x_, y_ = f_train.sample(700)
            sess.run(f_model.optimize, feed_dict={x:x_, y:y_, dropout:f_keep_prob, training:True})     
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([f_model.loss, f_model.acc], 
                                                 feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_valid.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        print("begin training classifier")
        train_time = time.time()
        for step in range(c_num_steps):        
            x_, y_ = c_train.sample(700)
            features_ = sess.run(f_model.features, feed_dict={x:x_, dropout:1.0, training:False})
            sess.run(c_model.optimize, feed_dict={features:features_, y:y_, x:x_, dropout:1.0, training:False})
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([c_model.loss, c_model.acc], feed_dict={features:features_, y:y_})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training classifier // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_valid, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_valid})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_test, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_test})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))

In [4]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.005, reg: 0.001, dropout: 0.6
	step 0: train loss 5.19729, train error 0.991429
	step 1000: train loss 4.93217, train error 0.954286
	step 2000: train loss 4.79396, train error 0.935714
	step 3000: train loss 4.57935, train error 0.91
	step 4000: train loss 4.37973, train error 0.894286
	step 5000: train loss 4.35247, train error 0.882857
	step 6000: train loss 4.25342, train error 0.874286
	step 7000: train loss 4.07276, train error 0.837143
	step 8000: train loss 4.08911, train error 0.845714
	step 9000: train loss 3.93279, train error 0.83
	step 10000: train loss 3.86516, train error 0.827143
	step 11000: train loss 3.82155, train error 0.798571
	step 12000: train loss 3.7176, train error 0.804286
	step 13000: train loss 3.66423, train error 0.778571
	step 14000: train loss 3.63825, train error 0.777143
	step 15000: train loss 3.58227, train error 0.777143
	step 16000: train loss 3.5453, train error 0.771429
	step 17000: train loss

In [5]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.005, reg: 0.001, dropout: 0.6
	step 0: train loss 1.64034, train error 0.535714
	step 1000: train loss 1.49851, train error 0.422857
	step 2000: train loss 1.42179, train error 0.368571
	step 3000: train loss 1.29873, train error 0.344286
	step 4000: train loss 1.27121, train error 0.342857
	step 5000: train loss 1.20242, train error 0.324286
	step 6000: train loss 1.22536, train error 0.347143
	step 7000: train loss 1.16909, train error 0.321429
	step 8000: train loss 1.15804, train error 0.335714
	step 9000: train loss 1.13623, train error 0.325714
	step 10000: train loss 1.11869, train error 0.315714
	step 11000: train loss 1.07868, train error 0.311429
	step 12000: train loss 1.02552, train error 0.29
	step 13000: train loss 1.09016, train error 0.321429
	step 14000: train loss 1.08744, train error 0.32
	step 15000: train loss 1.03725, train error 0.302857
	step 16000: train loss 1.04683, train error 0.318571
	step 17000: train lo

In [6]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.005, reg: 0.001, dropout: 0.6
	step 0: train loss 1.6621, train error 0.494286
	step 1000: train loss 1.50876, train error 0.461429
	step 2000: train loss 1.45701, train error 0.441429
	step 3000: train loss 1.32495, train error 0.37
	step 4000: train loss 1.30029, train error 0.404286
	step 5000: train loss 1.28049, train error 0.367143
	step 6000: train loss 1.23917, train error 0.374286
	step 7000: train loss 1.2244, train error 0.355714
	step 8000: train loss 1.17588, train error 0.337143
	step 9000: train loss 1.18741, train error 0.338571
	step 10000: train loss 1.16655, train error 0.337143
	step 11000: train loss 1.10445, train error 0.332857
	step 12000: train loss 1.11956, train error 0.351429
	step 13000: train loss 1.10379, train error 0.318571
	step 14000: train loss 1.07124, train error 0.311429
	step 15000: train loss 1.03232, train error 0.294286
	step 16000: train loss 1.03519, train error 0.311429
	step 17000: train 